In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pitching_23 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2023 Frontier League Pitching')
pitching_23['Name'] = pitching_23['Name'].str.replace('*', '')
pitching_23['Year'] = 2023

pitching_22 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2022 Frontier League Pitching')
pitching_22['Name'] = pitching_22['Name'].str.replace('*', '')
pitching_22['Year'] = 2022

pitching_21 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2021 Frontier League Pitching')
pitching_21['Name'] = pitching_21['Name'].str.replace('*', '')
pitching_21['Year'] = 2021

hitting_23 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2023 Frontier League Hitting')
hitting_23['Name'] = hitting_23['Name'].str.replace('*', '')
hitting_23['Year'] = 2023

hitting_22 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2022 Frontier League Hitting')
hitting_22['Name'] = hitting_22['Name'].str.replace('*', '')
hitting_22['Year'] = 2022

hitting_21 = pd.read_excel('../../../Downloads/FL 21-23.xlsx', '2021 Frontier League Hitting')
hitting_21['Name'] = hitting_21['Name'].str.replace('*', '')
hitting_21['Year'] = 2021

C:\Users\kellyjc\AppData\Local\Temp\ipykernel_23796\2318287953.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pitching_23['Name'] = pitching_23['Name'].str.replace('*', '')
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_23796\2318287953.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pitching_22['Name'] = pitching_22['Name'].str.replace('*', '')
C:\Users\kellyjc\AppData\Local\Temp\ipykernel_23796\2318287953.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pitching_21['Name'] = pitching_21['Name'].str.replace(

In [3]:
hitting_23['1B'] = hitting_23['H'] - (hitting_23['2B'] + hitting_23['3B'] + hitting_23['HR'])
hitting_22['1B'] = hitting_22['H'] - (hitting_22['2B'] + hitting_22['3B'] + hitting_22['HR'])
hitting_21['1B'] = hitting_21['H'] - (hitting_21['2B'] + hitting_21['3B'] + hitting_21['HR'])

In [4]:
def calculate_rates(df):
    df['BB/In'] = df['BB'] / df['IP']
    df['K/In'] = df['SO'] / df['IP']
    df['HR/In'] = df['HR'] / df['IP']
    df['W/In'] = df['W'] / df['IP']
    df['L/In'] = df['L'] / df['IP']
    df['S/In'] = df['SV'] / df['IP']
    df['ER/In'] = df['ER'] / df['IP']
    df['HBP/In'] = df['HBP'] / df['IP']
    
    return df

pitching_21 = calculate_rates(pitching_21)
pitching_22 = calculate_rates(pitching_22)
pitching_23 = calculate_rates(pitching_23)

In [5]:
def calculate_rates_hitters(df):
    df['BB/PA'] = df['BB'] / df['PA']
    df['K/PA'] = df['SO'] / df['PA']
    df['HR/PA'] = df['HR'] / df['PA']
    df['R/PA'] = df['R'] / df['PA']
    df['1B/PA'] = df['1B'] / df['PA']
    df['2B/PA'] = df['2B'] / df['PA']
    df['3B/PA'] = df['3B'] / df['PA']
    df['SB/PA'] = df['SB'] / df['PA']
    df['HBP/PA'] = df['HBP'] / df['PA']
    df['AB/PA'] = df['AB'] / df['PA']
    df['SF/PA'] = df['SF'] / df['PA']
    
    return df

hitting_21 = calculate_rates_hitters(hitting_21)
hitting_22 = calculate_rates_hitters(hitting_22)
hitting_23 = calculate_rates_hitters(hitting_23)

In [6]:
pitching = pd.concat([pitching_23, pitching_22, pitching_21])
hitting = pd.concat([hitting_23, hitting_22, hitting_21])

In [7]:
pitching_weights = {2023: 3, 2022: 2, 2021: 1}
hitting_weights = {2023: 5, 2022: 4, 2021: 3}

In [8]:
def calculate_pitching_metrics(group):
    divisor = 0
    for year, group_data in group.groupby('Year'):
        divisor += pitching_weights[year]
        
    result = {
        'K/In': (group['K/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'BB/In': (group['BB/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'HR/In': (group['HR/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'W/In': (group['W/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'L/In': (group['L/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'S/In': (group['S/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'ER/In': (group['ER/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum(),
        'HBP/In': (group['HBP/In'] * group.groupby('Year')['Year'].transform(lambda x: pitching_weights[int(x)] / divisor)).sum()
    }

    return pd.Series(result, index=['K/In', 'BB/In', 'HR/In', 'W/In', 'L/In', 'S/In', 'ER/In', 'HBP/In'])

pitcher_proj = pitching.groupby('Name').apply(calculate_pitching_metrics).reset_index()
pitcher_proj['IP'] = pitching.groupby('Name')['IP'].sum().values

In [9]:
def calculate_hitting_metrics(group):
    divisor = 0
    for year, group_data in group.groupby('Year'):
        divisor += hitting_weights[year]
        
    result = {
        'K/PA': (group['K/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'BB/PA': (group['BB/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'HR/PA': (group['HR/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        '1B/PA': (group['1B/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(), 
        '2B/PA': (group['2B/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        '3B/PA': (group['3B/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'SB/PA': (group['SB/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'HBP/PA': (group['HBP/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'R/PA': (group['R/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'AB/PA': (group['AB/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum(),
        'SF/PA': (group['SF/PA'] * group.groupby('Year')['Year'].transform(lambda x: hitting_weights[int(x)] / divisor)).sum()
    }

    return pd.Series(result, index=['K/PA', 'BB/PA', 'HR/PA', 'SB/PA', '1B/PA', '2B/PA', '3B/PA', 'HBP/PA', 'R/PA', 'SF/PA', 'AB/PA'])

hitter_proj = hitting.groupby('Name').apply(calculate_hitting_metrics).reset_index()
hitter_proj['PA'] = hitting.groupby('Name')['PA'].sum().values

In [10]:
hitter_proj.PA.mean(), pitcher_proj.IP.mean()

(411.21973094170403, 95.4429824561404)

In [11]:
avg_Kp = pitcher_proj['K/In'].mean()
avg_BBp = pitcher_proj['BB/In'].mean()
avg_HRp = pitcher_proj['HR/In'].mean()
avg_Wp = pitcher_proj['W/In'].mean()
avg_Lp = pitcher_proj['L/In'].mean()
avg_Sp = pitcher_proj['S/In'].mean()
avg_ERp = pitcher_proj['ER/In'].mean()
avg_HBPp = pitcher_proj['ER/In'].mean()

mask = pitcher_proj['IP'] < 100

pitcher_proj.loc[mask, 'K/In'] = (pitcher_proj.loc[mask, 'K/In'] * pitcher_proj.loc[mask, 'IP'] +
                                (100 - pitcher_proj.loc[mask, 'IP']) * avg_Kp) / 100

pitcher_proj.loc[mask, 'BB/In'] = (pitcher_proj.loc[mask, 'BB/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_BBp) / 100

pitcher_proj.loc[mask, 'HR/In'] = (pitcher_proj.loc[mask, 'HR/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_HRp) / 100

pitcher_proj.loc[mask, 'W/In'] = (pitcher_proj.loc[mask, 'W/In'] * pitcher_proj.loc[mask, 'IP'] +
                                (100 - pitcher_proj.loc[mask, 'IP']) * avg_Wp) / 100

pitcher_proj.loc[mask, 'L/In'] = (pitcher_proj.loc[mask, 'L/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_Lp) / 100

pitcher_proj.loc[mask, 'S/In'] = (pitcher_proj.loc[mask, 'S/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_Sp) / 100

pitcher_proj.loc[mask, 'ER/In'] = (pitcher_proj.loc[mask, 'ER/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_ERp) / 100

pitcher_proj.loc[mask, 'HBP/In'] = (pitcher_proj.loc[mask, 'HBP/In'] * pitcher_proj.loc[mask, 'IP'] +
                                 (100 - pitcher_proj.loc[mask, 'IP']) * avg_HBPp) / 100

pitcher_proj['Reliability'] = 1 - (100 - pitcher_proj['IP']).clip(0) / 100

In [12]:
avg_Kp = hitter_proj['K/PA'].mean()
avg_BBp = hitter_proj['BB/PA'].mean()
avg_HRp = hitter_proj['HR/PA'].mean()
avg_s = hitter_proj['1B/PA'].mean()
avg_d = hitter_proj['2B/PA'].mean()
avg_t = hitter_proj['3B/PA'].mean()
avg_sb = hitter_proj['SB/PA'].mean()
avg_r = hitter_proj['R/PA'].mean()
avg_hbp = hitter_proj['HBP/PA'].mean()
avg_ab = hitter_proj['AB/PA'].mean()
avg_sf = hitter_proj['SF/PA'].mean()

mask = hitter_proj['PA'] < 400

hitter_proj.loc[mask, 'K/PA'] = (hitter_proj.loc[mask, 'K/PA'] * hitter_proj.loc[mask, 'PA'] +
                                (400 - hitter_proj.loc[mask, 'PA']) * avg_Kp) / 400

hitter_proj.loc[mask, 'BB/PA'] = (hitter_proj.loc[mask, 'BB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_BBp) / 400

hitter_proj.loc[mask, 'HR/PA'] = (hitter_proj.loc[mask, 'HR/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_HRp) / 400

hitter_proj.loc[mask, '1B/PA'] = (hitter_proj.loc[mask, '1B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_s) / 400

hitter_proj.loc[mask, '2B/PA'] = (hitter_proj.loc[mask, '2B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_d) / 400

hitter_proj.loc[mask, '3B/PA'] = (hitter_proj.loc[mask, '3B/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_t) / 400

hitter_proj.loc[mask, 'SB/PA'] = (hitter_proj.loc[mask, 'SB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_sb) / 400

hitter_proj.loc[mask, 'R/PA'] = (hitter_proj.loc[mask, 'R/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_r) / 400

hitter_proj.loc[mask, 'HBP/PA'] = (hitter_proj.loc[mask, 'HBP/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_hbp) / 400

hitter_proj.loc[mask, 'AB/PA'] = (hitter_proj.loc[mask, 'AB/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_ab) / 400

hitter_proj.loc[mask, 'SF/PA'] = (hitter_proj.loc[mask, 'SF/PA'] * hitter_proj.loc[mask, 'PA'] +
                                 (400 - hitter_proj.loc[mask, 'PA']) * avg_sf) / 400

hitter_proj['Reliability'] = 1 - (400 - hitter_proj['PA']).clip(0) / 400

In [13]:
projections_24 = pitching_23[['Name', 'Age', 'Tm']].merge(pitcher_proj, on='Name', how='left')
projections_hit_24 = hitting_23[['Name', 'Age', 'Tm']].merge(hitter_proj, on='Name', how='left')

In [14]:
for p in projections_24.Name.unique():
    p_data = pitching_22[pitching_22['Name'] == p]
    ip_23 = pitching_23[pitching_23['Name'] == p].IP.values[0]
    if len(p_data) == 0:
        proj_ip = .5 * ip_23 + 30
    else: 
        proj_ip = .5 * ip_23 + .1 * p_data['IP'].values[0] + 30
    projections_24.loc[projections_24['Name'] == p, 'IP'] = proj_ip

In [15]:
for h in projections_hit_24['Name'].unique():
    h_data = hitting_22[hitting_22['Name'] == h]
    pa_23 = hitting_23[hitting_23['Name'] == h]['PA'].values[0]
    if len(h_data) == 0:
        proj_pa = 0.5 * pa_23 + 100
    else:
        proj_pa = 0.5 * pa_23 + 0.1 * h_data['PA'].values[0] + 100
    projections_hit_24.loc[projections_hit_24['Name'] == h, 'PA'] = proj_pa

In [16]:
projections_24['Age'] = pd.to_numeric(projections_24['Age'], errors='coerce')
projections_24['Age'] += 1
projections_24.loc[projections_24['Age'] >= 29, 'AgeAdj'] = 1 / (1 + (((projections_24['Age'] - 29) - 29) * 0.003))
projections_24.loc[projections_24['Age'] < 29, 'AgeAdj'] = 1 + ((29 - (projections_24['Age'] - 29)) * 0.006)

In [17]:
projections_hit_24['Age'] = pd.to_numeric(projections_hit_24['Age'], errors='coerce')
projections_hit_24['Age'] += 1
projections_hit_24.loc[projections_hit_24['Age'] >= 29, 'AgeAdj'] = 1 / (1 + (((projections_hit_24['Age'] - 29) - 29) * 0.003))
projections_hit_24.loc[projections_hit_24['Age'] < 29, 'AgeAdj'] = 1 + ((29 - (projections_hit_24['Age'] - 29)) * 0.006)

In [18]:
projections_24['IP'] *= projections_24['AgeAdj']
projections_hit_24['PA'] *= projections_hit_24['AgeAdj']

In [19]:
projections_24 = projections_24.dropna(subset=['IP'])
projections_hit_24 = projections_hit_24.dropna(subset=['PA'])

In [20]:
projections_24['K'] = projections_24['K/In'] * projections_24['IP']
projections_24['BB'] = projections_24['BB/In'] * projections_24['IP']
projections_24['HR'] = projections_24['HR/In'] * projections_24['IP']
projections_24['W'] = projections_24['W/In'] * projections_24['IP']
projections_24['L'] = projections_24['L/In'] * projections_24['IP']
projections_24['S'] = projections_24['S/In'] * projections_24['IP']
projections_24['ER'] = projections_24['ER/In'] * projections_24['IP']
projections_24['HBP'] = projections_24['HBP/In'] * projections_24['IP']

projections_24['K'] = projections_24['K'].apply(math.floor)
projections_24['BB'] = projections_24['BB'].apply(math.floor)
projections_24['HR'] = projections_24['HR'].apply(math.floor)
projections_24['W'] = projections_24['W'].apply(math.floor)
projections_24['L'] = projections_24['L'].apply(math.floor)
projections_24['S'] = projections_24['S'].apply(math.floor)
projections_24['ER'] = projections_24['ER'].apply(math.floor)
projections_24['HBP'] = projections_24['HBP'].apply(math.floor)

In [21]:
projections_hit_24['SB'] = projections_hit_24['SB/PA'] * projections_hit_24['PA']
projections_hit_24['HR'] = projections_hit_24['HR/PA'] * projections_hit_24['PA']
projections_hit_24['K'] = projections_hit_24['K/PA'] * projections_hit_24['PA']
projections_hit_24['BB'] = projections_hit_24['BB/PA'] * projections_hit_24['PA']
projections_hit_24['1B'] = projections_hit_24['1B/PA'] * projections_hit_24['PA']
projections_hit_24['2B'] = projections_hit_24['2B/PA'] * projections_hit_24['PA']
projections_hit_24['3B'] = projections_hit_24['3B/PA'] * projections_hit_24['PA']
projections_hit_24['SO'] = projections_hit_24['K/PA'] * projections_hit_24['PA']
projections_hit_24['R'] = projections_hit_24['R/PA'] * projections_hit_24['PA']
projections_hit_24['SB'] = projections_hit_24['SB/PA'] * projections_hit_24['PA']
projections_hit_24['HBP'] = projections_hit_24['HBP/PA'] * projections_hit_24['PA']
projections_hit_24['SF'] = projections_hit_24['SF/PA'] * projections_hit_24['PA']
projections_hit_24['AB'] = projections_hit_24['AB/PA'] * projections_hit_24['PA']

projections_hit_24['K'] = projections_hit_24['K'].apply(math.floor)
projections_hit_24['BB'] = projections_hit_24['BB'].apply(math.floor)
projections_hit_24['HR'] = projections_hit_24['HR'].apply(math.floor)
projections_hit_24['SB'] = projections_hit_24['SB'].apply(math.floor)
projections_hit_24['1B'] = projections_hit_24['1B'].apply(math.floor)
projections_hit_24['2B'] = projections_hit_24['2B'].apply(math.floor)
projections_hit_24['3B'] = projections_hit_24['3B'].apply(math.floor)
projections_hit_24['SO'] = projections_hit_24['SO'].apply(math.floor)
projections_hit_24['R'] = projections_hit_24['R'].apply(math.floor)
projections_hit_24['SB'] = projections_hit_24['SB'].apply(math.floor)
projections_hit_24['HBP'] = projections_hit_24['HBP'].apply(math.floor)
projections_hit_24['AB'] = projections_hit_24['AB'].apply(math.floor)
projections_hit_24['SF'] = projections_hit_24['SF'].apply(math.floor)

In [22]:
woba_vals = {'BB': 0.696, 'HBP': 0.726, '1B': 0.883, '2B': 1.244, '3B': 1.569, 'HR': 2.004, 'SB': 0.200, 'CS': -0.422}
def calculate_woba(df):
    woba = (
        (woba_vals['BB'] * df['BB'] +
        woba_vals['HBP'] * df['HBP'] +
        woba_vals['1B'] * df['1B'] +
        woba_vals['2B'] * df['2B'] +
        woba_vals['3B'] * df['3B'] +
        woba_vals['HR'] * df['HR'] +
        woba_vals['SB'] * df['SB'])  
        / (df['AB'] + df['BB'] + df['SF'] + df['HBP']))
    return woba

def calculate_babip(df):
    babip = (
        (df['H'] - df['HR']) / (df['AB'] - df['K'] - df['HR'] + df['SF']))
    return babip

In [23]:
projections_hit_24['H'] = projections_hit_24['1B'] + projections_hit_24['2B'] + projections_hit_24['3B'] + projections_hit_24['HR']

projections_hit_24['wOBA'] = calculate_woba(projections_hit_24)
projections_hit_24['BAbip'] = calculate_babip(projections_hit_24)

woba_scale = 1.204
projections_hit_24['wRAA'] = ((projections_hit_24['wOBA'] - projections_hit_24['wOBA'].mean()) / woba_scale) * projections_hit_24.PA
projections_hit_24['OBP'] = (projections_hit_24['H'] + projections_hit_24['HBP']) / projections_hit_24['PA']
projections_hit_24['BA'] = projections_hit_24['H'] / (projections_hit_24['AB'])
projections_hit_24['SLG'] = (projections_hit_24['1B'] * 1 + projections_hit_24['2B'] * 2 + projections_hit_24['3B'] * 3 + projections_hit_24['HR'] * 4) / (projections_hit_24['AB'])
projections_hit_24['bat-WAR'] = projections_hit_24['wRAA'] / 10

In [25]:
def calculate_fip(df):
    f_constant = df.ERA.mean() - (((13 * df.HR.sum()) + (3 * (df.BB.sum() + df.HBP.sum())) - (2 * df.K.sum())) / df.IP.sum())
    return f_constant + (((13 * df.HR) + (3 * df.BB) - (2 * df.K)) / df.IP)

projections_24['ERA'] = 9 * projections_24['ER'] / projections_24['IP']
projections_24['FIP'] = calculate_fip(projections_24)
projections_24['BB/9'] = projections_24['BB'] / 9
projections_24['K/9'] = projections_24['K'] / 9
projections_24['K/BB'] = projections_24['K'] / projections_24['BB']

In [26]:
projections_24 = projections_24[['Name', 'Age', 'Tm', 'IP', 'K', 'BB', 'HR', 'ERA', 'FIP', 'BB/9', 'K/9', 'K/BB', 'Reliability']]

In [27]:
projections_hit_24 = projections_hit_24[['Name', 'Age', 'Tm', '1B', '2B', '3B', 'HR', 'SO', 'SB', 'BB', 'HBP', 'wOBA', 'wRAA', 'BA', 'SLG', 'OBP', 'BAbip', 'bat-WAR', 'Reliability']]

In [30]:
projections_24.to_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/2024 Pitcher Projections.csv')

In [31]:
projections_hit_24.to_csv('../../../Documents/GitHub/Frontier-League-Savant/csvs/2024 Hitter Projections.csv')

In [32]:
projections_24

,Name,Age,Tm,IP,K,BB,HR,ERA,FIP,BB/9,K/9,K/BB,Reliability
0,Cole Cook,26.0,JOL,131.370320,124,28,4,2.603328,2.398308,3.111111,13.777778,4.428571,1.000
1,Joey Gonzalez,27.0,GAT,111.033320,111,46,17,4.458121,4.484724,5.111111,12.333333,2.413043,1.000
2,Turner Larkins,28.0,JOL,103.368000,73,28,7,4.092176,3.531411,3.111111,8.111111,2.607143,1.000
3,Collin Sullivan,26.0,GAT,101.379600,124,32,7,2.663258,2.649161,3.555556,13.777778,3.875000,1.000
4,Steven Fuentes,27.0,QUE,90.136000,58,32,5,2.496228,3.750106,3.555556,6.444444,1.812500,0.920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Cam Aufderheide,27.0,JOL,105.637020,78,45,8,4.345068,4.036569,5.000000,8.666667,1.733333,1.000
96,Jon Beymer,25.0,EVN,70.682000,75,34,3,3.947257,3.123529,3.777778,8.333333,2.205882,0.580
97,Garrison Bryant,25.0,TRC,79.726900,61,29,13,5.531383,4.931598,3.222222,6.777778,2.103448,0.731
98,Stephen Chamblee,28.0,LEC,77.408000,59,23,13,5.929620,4.801087,2.555556,6.555556,2.565217,0.712
